# Capacity Expansion


:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).
Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install -q pypsa pandas highspy "plotly<6"
```
:::

**In this tutorial, we want to build a replica of [model.energy](https://model.energy).** This tool calculates the cost of meeting a constant electricity demand from a combination of wind power, solar power and storage for different regions of the world. We deviate from [model.energy](https://model.energy) by including electricity demand profiles rather than a constant electricity demand.

:::{note}
See also https://model.energy.
:::

## Problem Extension

Review the problem formulation of the electricity market model from the previous tutorial. Below you can find an adapted version
where the capacity limits have been promoted to **decision variables** with corresponding terms
in the *objective function* and *new constraints for their expansion limits* (e.g. wind and solar potentials). This is known as **capacity expansion problem**.

\begin{equation*}
    \min_{g,e,f,G,E,F} \quad \sum_{i,s,t} w_t o_{s} g_{i,s,t} + \sum_{i,s} c_sG_{i,s}  + c_{r,\text{dis/charge}}G_{i,r, \text{dis/charge}} +   c_{r}E_{i,r}  + c_\ell F_{\ell}
  \end{equation*}
such that
  \begin{align*}
    d_{i,t} &= \sum_s g_{i,s,t}  - \sum_\ell K_{i\ell} f_{\ell,t}   & \text{energy balance} \\
    0 &\leq g_{i,s,t} \leq \hat{g}_{i,s,t} G_{i,s} & \text{generator limits}\\
    0 & \leq g_{i,r,t,\text{dis/charge}} \leq G_{i,r,\text{dis/charge}}& \text{storage dis/charge limits} \\
    0 & \leq e_{i,r,t} \leq E_{r} & \text{storage energy limits} \\ 
    e_{i,r,t} &= \eta^0_{i,r,t} e_{i,r,t-1} + \eta^1_{r}g_{i,r,t,\text{charge}} -  \frac{1}{\eta^2_{r}} g_{i,r,t,\text{discharge}} & \text{storage consistency} \\
    -F_\ell &\leq f_{\ell,t} \leq F_\ell  & \text{line limits} \\
    0 &= \sum_\ell C_{\ell c} x_\ell f_{\ell,t} & \text{KVL} \\
        \underline{G}_{i,s} & \leq G_{i,s} \leq \overline{G}_{i,s} & \text{generator capacity expansion limits} \\
    \underline{G}_{i,r, \text{dis/charge}} & \leq G_{i,r, \text{dis/charge}} \leq \overline{G}_{i,r, \text{dis/charge}} & \text{storage power capacity expansion limits} \\
    \underline{E}_{i,r} & \leq E_{i,r} \leq \overline{E}_{i,r} & \text{storage energy expansion limits} \\
    \underline{F}_{\ell} & \leq F_{\ell} \leq \overline{F}_{\ell} & \text{line capacity expansion limits}
  \end{align*}

**New decision variables for capacity expansion planning:**

- $G_{i,s}$ is the generator capacity at bus $i$, technology $s$,
- $F_{\ell}$ is the transmission capacity of line $\ell$,
- $G_{i,r,\text{dis-/charge}}$ denotes the charge and discharge capacities of storage unit $r$ at bus $i$,
- $E_{i,r}$ is the energy capacity of storage $r$ at bus $i$ and time step $t$.

**New parameters for capacity expansion planning:**

- $c_{\star}$ is the capital cost of technology $\star$ at bus $i$
- $w_t$ is the weighting of time step $t$ (e.g. number of hours it represents)
- $\underline{G}_\star, \underline{F}_\star, \underline{E}_\star$ are the minimum capacities per technology and location/connection.
- $\underline{G}_\star, \underline{F}_\star, \underline{E}_\star$ are the maximum capacities per technology and location.

:::{note}
For a full reference to the optimisation problem description, see the [PyPSA User Guide](https://pypsa--1250.org.readthedocs.build/en/1250/user-guide/optimization/overview/)
:::

## Package Imports

In [ ]:
import pypsa
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.offline as py
from plotly.subplots import make_subplots
from pypsa.common import annuity

pd.options.plotting.backend = "plotly"

## Capacity Factor & Load Time Series

We are also going to need some time series for wind, solar and load.

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/9toBssWEdaLgHzq/download/time-series.csv"
ts = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
ts.head(3)

We are also going to adapt the temporal resolution of the time series, e.g. sample only every other hour, to save some time:

In [ ]:
resolution = 3
ts = ts.resample(f"{resolution}h").first()
ts

## Model Initialisation

For building the model, we start again by initialising an empty network.

In [ ]:
n = pypsa.Network()

Then, we add a single bus...

In [ ]:
n.add("Bus", "electricity", carrier="electricity")

...and tell the `pypsa.Network` object `n` what the snapshots of the model will be using the utility function `n.set_snapshots()`.

In [ ]:
n.set_snapshots(ts.index)

In [ ]:
n.snapshots[:5]

The weighting of the snapshots (e.g. how many hours they represent, see $w_t$ in problem formulation above) can be set in `n.snapshot_weightings`.

In [ ]:
n.snapshot_weightings.loc[:, :] = resolution

In [ ]:
n.snapshot_weightings.head(3)

## Adding Components

Then, we add all the technologies we are going to include as carriers.

In [ ]:
carriers = [
    "wind",
    "solar",
    "OCGT",
    "hydrogen storage underground",
    "battery storage",
    "electricity",
]

n.add(
    "Carrier",
    carriers,
    color=["dodgerblue", "gold", "indianred", "magenta", "yellowgreen", "black"],
    co2_emissions=[0, 0, 0.2, 0, 0, 0],
);

Next, we add the demand time series to the model.

In [ ]:
n.add(
    "Load",
    "demand",
    bus="electricity",
    carrier="electricity",
    p_set=ts.load_mw,
);

Let's have a check whether the data was read-in correctly.

In [ ]:
n.loads_t.p_set.plot()

We are going to add one dispatchable generation technology to the model. This is an open-cycle gas turbine (OCGT) with CO$_2$ emissions of 0.2 t/MWh$_{th}$.

In [ ]:
n.add(
    "Generator",
    "OCGT",
    bus="electricity",
    carrier="OCGT",
    capital_cost=annuity(0.07, 30) * 500_000,
    marginal_cost=65,
    efficiency=0.4,
    p_nom_extendable=True,
)

Adding the variable renewable generators works almost identically, but we also need to supply the capacity factors to the model via the attribute `p_max_pu`.

In [ ]:
n.add(
    "Generator",
    "wind",
    bus="electricity",
    carrier="wind",
    p_max_pu=ts.wind_pu,
    capital_cost=annuity(0.07, 30) * 1_100_000,
    marginal_cost=1,
    p_nom_extendable=True,
);

In [ ]:
n.add(
    "Generator",
    "solar",
    bus="electricity",
    carrier="solar",
    p_max_pu=ts.pv_pu,
    capital_cost=annuity(0.05, 25) * 500_000,
    marginal_cost=0.5,
    p_nom_extendable=True,
);

So let's make sure the capacity factors are read-in correctly.

In [ ]:
n.generators_t.p_max_pu.loc["2019-03"].plot()

### Model Run

Then, we can already solve the model for the first time. At this stage, the model does not have any storage or emission limits implemented. It's going to look for the least-cost combination of variable renewables and the gas turbine to supply demand.

In [ ]:
n.optimize(log_to_console=False)

### Model Evaluation

The total system cost in billion Euros per year:

In [ ]:
n.objective / 1e9

The optimised capacities in GW:

In [ ]:
n.generators.p_nom_opt.div(1e3)  # GW

The energy balance by component in TWh:

In [ ]:
n.statistics.energy_balance().sort_values().div(1e6)  # TWh

While we get the objective value through `n.objective`, in many cases we want to know how the costs are distributed across the technologies. We can use the statistics module for this:

In [ ]:
(n.statistics.capex() + n.statistics.opex()).div(1e6)

Possibly, we are also interested in the total emissions:

In [ ]:
emissions = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
)  # t/h

In [ ]:
n.snapshot_weightings.generators @ emissions.sum(axis=1).div(1e6)  # Mt

In [ ]:
n.statistics.energy_balance.iplot()

## Adding Storage Units

Alright, but there are a few important components missing for a system with high shares of renewables? What about short-term storage options (e.g. batteries) and long-term storage options (e.g. hydrogen storage)? Let's add them, too.

First, the battery storage. We are going to assume a fixed energy-to-power ratio of 4 hours, i.e. if fully charged, the battery can discharge at full capacity for 4 hours.

For the capital cost, we have to factor in both the capacity and energy cost of the storage. We are also going to enforce a cyclic state-of-charge condition, i.e. the state of charge at the beginning of the optimisation period must equal the final state of charge.

In [ ]:
n.add(
    "StorageUnit",
    "battery storage",
    bus="electricity",
    carrier="battery storage",
    max_hours=4,
    capital_cost=annuity(0.07, 10) * 150_000 + 4 * annuity(0.07, 20) * 150_000,
    efficiency_store=0.95,
    efficiency_dispatch=0.95,
    p_nom_extendable=True,
    cyclic_state_of_charge=True,
);

Second, the hydrogen storage. This one is composed of an electrolysis to convert electricity to hydrogen, a fuel cell to re-convert hydrogen to electricity and underground storage (e.g. in salt caverns). We assume an energy-to-power ratio of 336 hours, such that this type of storage can be used for weekly balancing.

In [ ]:
capital_costs = (
    annuity(0.07, 25) * 1_500_000
    + annuity(0.07, 25) * 550_000
    + 336 * annuity(0.07, 100) * 2_000
)

n.add(
    "StorageUnit",
    "hydrogen storage underground",
    bus="electricity",
    carrier="hydrogen storage underground",
    max_hours=336,
    capital_cost=capital_costs,
    efficiency_store=0.65,
    efficiency_dispatch=0.45,
    p_nom_extendable=True,
    cyclic_state_of_charge=True,
);

Ok, lets run the again, now with storage, and see what's changed.

In [ ]:
n.optimize(log_to_console=False)

In [ ]:
n.statistics.optimal_capacity().div(1e3)  # GW

In [ ]:
n.statistics.energy_balance().sort_values().div(1e6)  # .sum()  # TWh

In [ ]:
pd.concat(
    {
        "capex": n.statistics.capex(),
        "opex": n.statistics.opex(),
    },
    axis=1,
).div(1e9).round(
    2
)  # bn€/a

In [ ]:
n.buses_t.marginal_price.sort_values(by="electricity", ascending=False).reset_index(
    drop=True
).plot(title="price duration curve [€/MWh]")

### Adding emission limits

Now, let's model a 100% renewable electricity system by adding a CO$_2$ emission limit as global constraint:

In [ ]:
n.add(
    "GlobalConstraint",
    "CO2Limit",
    carrier_attribute="co2_emissions",
    sense="<=",
    constant=0,
);

When we run the model now...

In [ ]:
n.optimize(log_to_console=False)

In [ ]:
n.statistics.optimal_capacity().div(1e3)  # GW

In [ ]:
n.statistics.energy_balance().sort_values().div(1e6)  # TWh

In [ ]:
pd.concat(
    {
        "capex": n.statistics.capex(),
        "opex": n.statistics.opex(),
    },
    axis=1,
).div(1e9).round(
    2
)  # bn€/a

In [ ]:
n.storage_units.p_nom_opt.div(1e3) * n.storage_units.max_hours  # GWh

In [ ]:
n.statistics.energy_balance.iplot()

In [ ]:
n.buses_t.marginal_price.sort_values(by="electricity", ascending=False).reset_index(
    drop=True
).plot(title="price duration curve [€/MWh]")

Finally, we will export this network so that we can build on it when adding further sectors (e.g. electric vehicles and heat pumps) in the next tutorial:

In [ ]:
n.export_to_netcdf("electricity-network.nc");

## Exercises

Explore how the model reacts to changing assumptions and available technologies. Here are a few inspirations, but choose in any order according to your interests:

- What if the model were rerun with assumptions for 2050?
- What if either hydrogen or battery storage cannot be expanded?
- What if you could either only build solar or only build wind?
- Vary the energy-to-power ratio of the hydrogen storage. What ratio leads to lowest costs?
- On [model.energy](https://model.energy), you can download capacity factors for onshore wind and solar for any region in the world. What changes?